In [65]:
from dotenv import load_dotenv
load_dotenv(override=True)
from openai import AsyncOpenAI

client = AsyncOpenAI()

tools = [
  {
    "type": "function",
    "function": {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA",
          },
          "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
        },
        "required": ["location"],
      },
    }
  }
]
messages = [{"role": "user", "content": "What's the weather like in Boston, Tokyo and Seoul today?"}]
stream = await client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=messages,
  tools=tools,
  tool_choice="auto",
  stream=True
)



message = {
    'role': 'assistant',
    'content': '',
    'tool_calls': []
}

async for chunk in stream:
    delta = chunk.choices[0].delta
    # Handle role updates
    if delta.role:
        message['role'] = delta.role
    
    # Handle content updates
    if delta.content:
        message['content'] += delta.content

    # Handle tool calls
    if delta.tool_calls:
        for tool_call in delta.tool_calls:
            call_index = tool_call.index
            if len(message['tool_calls']) == call_index:
                # new tool call started, so add a new empty object to the tool_calls list
                message['tool_calls'].append({'id': None, 'type': 'function', 'function': {'name': '', 'arguments': ''}})
            if tool_call.id:
                message['tool_calls'][call_index]['id'] = tool_call.id
            if tool_call.type:
                message['tool_calls'][call_index]['type'] = tool_call.type
            if tool_call.function.name:
                message['tool_calls'][call_index]['function']['name'] = tool_call.function.name
            if tool_call.function.arguments:
                message['tool_calls'][call_index]['function']['arguments'] += tool_call.function.arguments
    print(message)

message

{'role': 'assistant', 'content': '', 'tool_calls': []}
{'role': 'assistant', 'content': '', 'tool_calls': [{'id': 'call_RTDeC1bBR4cS8OepqxuOKVVf', 'type': 'function', 'function': {'name': 'get_current_weather', 'arguments': ''}}]}
{'role': 'assistant', 'content': '', 'tool_calls': [{'id': 'call_RTDeC1bBR4cS8OepqxuOKVVf', 'type': 'function', 'function': {'name': 'get_current_weather', 'arguments': '{"lo'}}]}
{'role': 'assistant', 'content': '', 'tool_calls': [{'id': 'call_RTDeC1bBR4cS8OepqxuOKVVf', 'type': 'function', 'function': {'name': 'get_current_weather', 'arguments': '{"locatio'}}]}
{'role': 'assistant', 'content': '', 'tool_calls': [{'id': 'call_RTDeC1bBR4cS8OepqxuOKVVf', 'type': 'function', 'function': {'name': 'get_current_weather', 'arguments': '{"location": "B'}}]}
{'role': 'assistant', 'content': '', 'tool_calls': [{'id': 'call_RTDeC1bBR4cS8OepqxuOKVVf', 'type': 'function', 'function': {'name': 'get_current_weather', 'arguments': '{"location": "Bosto'}}]}
{'role': 'assistan

{'role': 'assistant',
 'content': '',
 'tool_calls': [{'id': 'call_RTDeC1bBR4cS8OepqxuOKVVf',
   'type': 'function',
   'function': {'name': 'get_current_weather',
    'arguments': '{"location": "Boston", "unit": "celsius"}'}},
  {'id': 'call_4DKfUMKmpcDXVM3HDcewgcTx',
   'type': 'function',
   'function': {'name': 'get_current_weather',
    'arguments': '{"location": "Tokyo", "unit": "celsius"}'}},
  {'id': 'call_BoBHLvIueH1PPF07zbY4R2As',
   'type': 'function',
   'function': {'name': 'get_current_weather',
    'arguments': '{"location": "Seoul", "unit": "celsius"}'}}]}